<h2> Brady Kruse & Marcus Jordan </h2>

<h1> Nim Rules (for reference) </h1>

<h2> To Dos </h2>

1) Running Minimax, setting up scores, etc.
1.5) Heuristic

2) Put some gui in!

<h1> Nim Rules & Tree Setup </h1>

#Nim is a game w/ a collection of an arbitrary number of stacks & an arbitrary number of stones per stack
#Players take turns removing stones (they can remove as many as they want / can (no negative stones) or as a few as they want / can but HAVE to remove 1)
#So what I've done is thus:
#The Stack class acts as a stack
#nim is a starting state of the game, represented by an array of stacks.
#Node is simply a node containing a current state of nim (an array of stacks) that also links to its children and has its player status
#NimTree is a class containing, you guessed it, a tree of Nim Nodes. 

In [61]:
def childrenPopulate(nim):
    children = []
    stack_index = 0
    for each in nim:
        for i in range(1, each.size+1):
            temp_nim = copy.deepcopy(nim)
            temp_nim[stack_index].removeStone(i)
            children.append(Node(temp_nim, childrenPopulate(temp_nim)))
        stack_index += 1
    return children

def firstDepthChildren(nim, player):
    children = []
    stack_index = 0
    for stack in nim:
        for i in range(1, stack.size+1):
            temp_nim = copy.deepcopy(nim)
            temp_nim[stack_index].removeStone(i)
            children.append(Node(temp_nim, player= not player))
        stack_index += 1
    return children

In [62]:
def playerSet(node, role):
    node.player = role
    for child in node.children:
        playerSet(child, not node.player)

In [63]:
count = 0
def printAllNodes(node):
    global count
    children = node.children
    for child in children:
        printAllNodes(child)
    
    node.printNode()
    count += 1

In [64]:
class Stack:
    def __init__(self, size):
        self.size = size
        self.original_size = size
    
    def removeStone(self, amount):
        if amount <= self.size and amount != 0:
            self.size -= amount
            return True #Indicates a valid turn
        
        return False #Indicates an invalid turn

In [65]:
class Node: #For ease, TRUE is the MAX player and FALSE is the MIN player
    def __init__(self, state, children=None, parent=None, player=None): #Might need parent...but at the same time, do we even care about the parent? Would we use it?
        self.state = state
        self.parent = parent
        self.children = children
        self.player = None
        if children != None:
            for child in self.children:
                child.parent = self
    
    def printNode(self): #Have no idea why, on Ubuntu end = "" throws a syntax error? Something to do w/ how printing is handled
        print("Node: ")
        for stack in self.state: print(stack.size)#, end=" ")
        print("\n")
    
        print("Player: ")#, end ="")
        if (self.player):
            print("AI\n")
        else:
            print("PUNY HUMAN\n")

        if(self.parent != None):
            print("Parent: ")
            for stack in self.parent.state: print(stack.size)#, end = " ")
            print("\n")

        if(self.children != None):
            print("Children: ")
            for child in self.children:
                for stack in child.state: print(stack.size)#, end = " ")
                print("\n")
        
    def validTurn(self): #Check each stack, if all are 0, no valid turns are left & Return false. 
        dead_stacks = 0
        for stack in self.state:
            if stack.size == 0:
                dead_stacks+=1
        if(dead_stacks == len(self.state)):
            return False
        return True
        

In [66]:
class NimTree:
    def __init__(self, nim, player):
        self.nim = nim
        self.origin = Node(nim)
        self.origin.children = childrenPopulate(self.origin.state)
        self.win_count = 0
        playerSet(self.origin, player)
        
    def possibleWins(self, node):
        for child in node.children:
            self.possibleWins(child)
            
        if not node.validTurn() and not node.player:
            #print("WIN")
            #node.printNode()
            #print("--- turn end ---" + "\n")
            self.win_count += 1
        elif not node.validTurn() and node.player:
            #print("LOSS")
            #node.printNode()
            #print("--- turn end ---" + "\n")
            return
        else:
            return
            
    def countWins(self): 
        self.possibleWins(self.origin)
        return self.win_count
    

In [67]:
import copy
import math

In [68]:
nim = [Stack(3), Stack(3), Stack(3)]

In [69]:
for stack in nim: print(stack.size)

3
3
3


In [70]:
nim = [Stack(3), Stack(3), Stack(3)]
tree = NimTree(nim, False) 

In [71]:
#this prints the nim state of the origin node of the tree
for stack in tree.origin.state: print(stack.size)#, end = " ")
print("\n") 

3
3
3




In [72]:
#This prints the nim state of all of the children nodes of a node (in this case, origin)
for child in tree.origin.children:
    for stack in child.state: print(stack.size)#, end = " ")
    print("\n")

2
3
3


1
3
3


0
3
3


3
2
3


3
1
3


3
0
3


3
3
2


3
3
1


3
3
0




In [73]:
#Commented out because this takes 5ever to run
#We're getting some repeated nodes...I think this is how it should be though?
#count = 0
#printAllNodes(tree.origin)

#print(count)

<h1> MiniMax Tree </h1>

<p1> MARCUS TODO: Okay so the idea here is that win for MAX is denoted a 1 and the win for MIN is denoted a 0. Once we have that setup, we need to define some sort of heuristic for game AI to choose the odds of it getting a 1 more. We could probably just use a simple ratio here. Try to import what we have into this code and see what we get. </p1>

In [74]:
# https://www.geeksforgeeks.org/minimax-algorithm-in-game-theory-set-1-introduction/
# A simple Python3 program to find 
# maximum score that 
# maximizing player can get
  
def minimax (curDepth, nodeIndex, 
             maxTurn, scores,  
             targetDepth): 
  
    # base case : targetDepth reached 
    if (curDepth == targetDepth):  
        return scores[nodeIndex] 
      
    if (maxTurn): 
        return max(minimax(curDepth + 1, nodeIndex * 2,  
                    False, scores, targetDepth),  
                   minimax(curDepth + 1, nodeIndex * 2 + 1,  
                    False, scores, targetDepth)) 
      
    else: 
        return min(minimax(curDepth + 1, nodeIndex * 2,  
                     True, scores, targetDepth),  
                   minimax(curDepth + 1, nodeIndex * 2 + 1,  
                     True, scores, targetDepth)) 
      
# Driver code 
scores = [3, 5, 2, 9, 12, 5, 23, 23] 
  
treeDepth = math.log(len(scores), 2) 
  
print("The optimal value is : ")#, end = "") 
print(minimax(0, 0, True, scores, treeDepth)) 
  
# This code is contributed 
# by rootshadow 

The optimal value is : 
12


In [75]:
custom_nim = [Stack(1), Stack(1), Stack(1)]
custom_tree = NimTree(custom_nim, True)
printAllNodes(custom_tree.origin)

Node: 
0
0
0


Player: 
PUNY HUMAN

Parent: 
0
0
1


Children: 
Node: 
0
0
1


Player: 
AI

Parent: 
0
1
1


Children: 
0
0
0


Node: 
0
0
0


Player: 
PUNY HUMAN

Parent: 
0
1
0


Children: 
Node: 
0
1
0


Player: 
AI

Parent: 
0
1
1


Children: 
0
0
0


Node: 
0
1
1


Player: 
PUNY HUMAN

Children: 
0
0
1


0
1
0


Node: 
0
0
0


Player: 
PUNY HUMAN

Parent: 
0
0
1


Children: 
Node: 
0
0
1


Player: 
AI

Parent: 
1
0
1


Children: 
0
0
0


Node: 
0
0
0


Player: 
PUNY HUMAN

Parent: 
1
0
0


Children: 
Node: 
1
0
0


Player: 
AI

Parent: 
1
0
1


Children: 
0
0
0


Node: 
1
0
1


Player: 
PUNY HUMAN

Children: 
0
0
1


1
0
0


Node: 
0
0
0


Player: 
PUNY HUMAN

Parent: 
0
1
0


Children: 
Node: 
0
1
0


Player: 
AI

Parent: 
1
1
0


Children: 
0
0
0


Node: 
0
0
0


Player: 
PUNY HUMAN

Parent: 
1
0
0


Children: 
Node: 
1
0
0


Player: 
AI

Parent: 
1
1
0


Children: 
0
0
0


Node: 
1
1
0


Player: 
PUNY HUMAN

Children: 
0
1
0


1
0
0


Node: 
1
1
1


Player: 
AI

Children: 
0
1


In [76]:
print(custom_tree.countWins())

6


<h1> User Input & Test Playing </h1>

In [77]:
def AI(node):
    node_win_count = 0
    next_move = None
    node_children = firstDepthChildren(node.state, node.player)
    for child in node_children:
        #child.printNode()
        child_tree = NimTree(child.state, child.player)
        child_wins = child_tree.countWins()
        if child_wins > node_win_count: #Heuristic based on depth HERE
            node_win_count = child_wins
            next_move = child
    
    return node_win_count, next_move
    

In [78]:
custom_nim = [Stack(1), Stack(1), Stack(1)]
custom_nim_node = Node(custom_nim, player=True)
AI_wins, AI_next_move = AI(custom_nim_node)
print("AI Wins: ", AI_wins, "\n\nNext Move:\n")

AI Wins:  2 

Next Move:



In [79]:
def gameLoop(nim, player):
    print("\t   GAME START\nMAY NIMROD HAVE MERCY ON YOUR SOUL\n----------------------------------\n")
    current_state = Node(nim)
    current_state.player = player
    current_state.printNode()
    print("----------------------------------\n")
    
    while(True):
        if(current_state.player): #AI's Turn
            AI_wins, AI_next_move = AI(current_state)
            if(AI_wins == 0):
                print("You Won! Somehow...")
                break
                
            if(not AI_next_move.validTurn()):
                current_state = Node([Stack(0), Stack(0), Stack(0)])
                print("AI Moves To")
                current_state.printNode()
                print("THE MIGHTY NIMROD BESTED YOU AGAIN")
                break
    
            current_state = AI_next_move
            
            print("AI Moves To:")
            current_state.printNode()
            print("----------------END TURN------------------\n")
            
        else: #Your turn!
            print("It's Your Turn")
            for stack in current_state.state:
                print("Stack: ", current_state.state.index(stack), ": ", stack.size)
            
            stack_choice = int(input("Stack: "))
            remove_amount = int(input("Stone #: "))
            while(stack_choice > len(current_state.state)-1):
                stack_choice = int(input("Pick Valid Stack: "))
                remove_amount = int(input("Stone #: "))
            while(not current_state.state[stack_choice].removeStone(remove_amount)):
                print("INVALID CHOICE! THIS ANGERS NIMROD!")
                stack_choice = int(input("Pick Valid Stack: "))
                remove_amount = int(input("Stone #: "))
            
            print("\nYou Move To:")
            current_state.player = not current_state.player
            current_state.printNode()
            print("----------------END TURN------------------\n")

In [80]:
test_nim = [Stack(3), Stack(3), Stack(3)] #ERROR: AI will skip it's "losing" turn as the algorithm returns 0.
gameLoop(test_nim, False)

	   GAME START
MAY NIMROD HAVE MERCY ON YOUR SOUL
----------------------------------

Node: 
3
3
3


Player: 
PUNY HUMAN

----------------------------------

It's Your Turn
Stack:  0 :  3
Stack:  1 :  3
Stack:  2 :  3


KeyboardInterrupt: 

<h1> Heuristic Creation</h1>